In [ ]:
import re

import networkx as nx
import pygraphviz as pgv


def dot_to_dict_with_positions(dot_graph: str):
    # Patterns to extract nodes and edges
    node_pattern = re.compile(
        r'(\w+)\s*\[label="\{\{<(\w+)> (\w+)\|<(\w+)> (\w+)\} \| (\w+: \w+) \| \{<(\w+)> (\w+)\|<(\w+)> (\w+)\}\}"]'
    )
    edge_pattern = re.compile(r"(\w+):(\w+)\s*--\s*(\w+):(\w+);")

    matches = node_pattern.findall(dot_graph)
    edge_matches = edge_pattern.findall(dot_graph)

    # Use pygraphviz to get positions
    A = pgv.AGraph(string=dot_graph)
    A.layout(prog="dot")

    nodes = {}
    edges = []

    for match in matches:
        node_name = match[0]
        node_label = match[5]
        ports = [match[1], match[3], match[7], match[9]]

        node_pos = A.get_node(node_name).attr["pos"]
        node_pos = tuple(map(float, node_pos.split(",")))
        nodes[node_label] = {"pos": node_pos, "size": 3000, "color": "green"}

        # Relative positions for ports around the main node
        relative_positions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for i, port in enumerate(ports):
            port_label = f"{node_name}:{port}"
            port_pos = (
                node_pos[0] + relative_positions[i][0],
                node_pos[1] + relative_positions[i][1],
            )
            nodes[port_label] = {"pos": port_pos, "size": 1500, "color": "skyblue"}
            edges.append((node_label, port_label))

    # Add edges between quasi-nodes
    for edge in edge_matches:
        node1, port1, node2, port2 = edge
        edge_from = f"{node1}:{port1}"
        edge_to = f"{node2}:{port2}"
        edges.append((edge_from, edge_to))

    return {"nodes": nodes, "edges": edges}


dot_string = """
graph cascaded_mzi_circuit {
  rankdir=LR;
  node [shape=record];

  C1 [label="{{<o2> o2|<o1> o1} | C1: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o1> o1|<o2> o2} | C2: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];

  C1:o3 -- C2:o1;
  C1:o4 -- C2:o2;
}
"""
from pprint import pprint

graph_dict = dot_to_dict_with_positions(dot_string)
pprint(graph_dict)


G = nx.Graph()
for node, attr in graph_dict["nodes"].items():
    G.add_node(node, **attr)
G.add_edges_from(graph_dict["edges"])

pos = {node: data["pos"] for node, data in G.nodes(data=True)}
sizes = [data["size"] for node, data in G.nodes(data=True)]
colors = [data["color"] for node, data in G.nodes(data=True)]

nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=sizes,
    node_color=colors,
    font_size=12,
    font_weight="bold",
)

nx.is_planar(G)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Define the graph in a dictionary
graph_dict = {
    "nodes": {
        "C": {"pos": (0, 0), "size": 3000, "color": "green"},
        "o1": {"pos": (1, 1), "size": 1500, "color": "skyblue"},
        "o2": {"pos": (1, -1), "size": 1500, "color": "skyblue"},
        "o3": {"pos": (-1, 1), "size": 1500, "color": "skyblue"},
        "o4": {"pos": (-1, -1), "size": 1500, "color": "skyblue"},
    },
    "edges": [("C", "o1"), ("C", "o2"), ("C", "o3"), ("C", "o4")],
}

# Create the graph
G = nx.Graph()

# Add nodes with attributes
for node, attr in graph_dict["nodes"].items():
    G.add_node(node, **attr)

# Add edges
G.add_edges_from(graph_dict["edges"])

# Extract positions, sizes, and colors
pos = {node: data["pos"] for node, data in G.nodes(data=True)}
sizes = [data["size"] for node, data in G.nodes(data=True)]
colors = [data["color"] for node, data in G.nodes(data=True)]

# Draw the graph
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=sizes,
    node_color=colors,
    font_size=12,
    font_weight="bold",
)
plt.show()